# Data Science Project - Analysis of Real Estate Data in the US

# Introduction

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
re_data = pd.read_csv('real_estate_db.csv', encoding = 'ISO-8859–1')

In [3]:
re_data.shape

(39030, 80)

In [4]:
re_data.columns

Index(['UID', 'BLOCKID', 'SUMLEVEL', 'COUNTYID', 'STATEID', 'state',
       'state_ab', 'city', 'place', 'type', 'primary', 'zip_code', 'area_code',
       'lat', 'lng', 'ALand', 'AWater', 'pop', 'male_pop', 'female_pop',
       'rent_mean', 'rent_median', 'rent_stdev', 'rent_sample_weight',
       'rent_samples', 'rent_gt_10', 'rent_gt_15', 'rent_gt_20', 'rent_gt_25',
       'rent_gt_30', 'rent_gt_35', 'rent_gt_40', 'rent_gt_50',
       'universe_samples', 'used_samples', 'hi_mean', 'hi_median', 'hi_stdev',
       'hi_sample_weight', 'hi_samples', 'family_mean', 'family_median',
       'family_stdev', 'family_sample_weight', 'family_samples',
       'hc_mortgage_mean', 'hc_mortgage_median', 'hc_mortgage_stdev',
       'hc_mortgage_sample_weight', 'hc_mortgage_samples', 'hc_mean',
       'hc_median', 'hc_stdev', 'hc_samples', 'hc_sample_weight',
       'home_equity_second_mortgage', 'second_mortgage', 'home_equity', 'debt',
       'second_mortgage_cdf', 'home_equity_cdf', 'debt_cdf', '

# Part 1. Cleaning

## The dataset contains 79 columns, so we will take only the most interesting ones that we find useful for our data exploration.

In [5]:
re_data = re_data[['state','city','zip_code','lat','lng','pop','male_pop','female_pop','rent_mean','rent_median',
              'hc_mean', 'hc_mortgage_mean','hc_mortgage_median', 'hi_mean','hi_median','family_mean',
              'home_equity_second_mortgage', 'second_mortgage', 'home_equity',
              'debt', 'hs_degree', 'male_age_mean', 'female_age_mean', 'pct_own','married'
             ]]

In [6]:
re_data.isna().sum().nlargest(20)

hc_mean                        890
hc_mortgage_mean               841
hc_mortgage_median             841
home_equity_second_mortgage    677
second_mortgage                677
home_equity                    677
debt                           677
rent_mean                      462
rent_median                    462
family_mean                    434
hi_mean                        390
hi_median                      390
pct_own                        390
female_age_mean                302
hs_degree                      275
married                        275
male_age_mean                  273
state                            0
city                             0
zip_code                         0
dtype: int64

In [7]:
re_data.dropna(inplace=True)

## We will drop Puerto Rico State, as it's economy is different from other states

In [8]:
re_data = re_data[re_data.state != 'Puerto Rico']

## Drop places with mean rent < 500 USD. Not realistic or really small village areas

In [9]:
re_data = re_data[re_data.rent_mean > 500]

In [10]:
re_data.rent_mean.nsmallest()

32076    500.01272
30717    500.02145
17906    500.13603
32069    500.38186
11155    500.54241
Name: rent_mean, dtype: float64

## We found not correct values where rent_mean is 3962.34229 due to bad record

In [11]:
re_data.groupby(['state'])[ 'rent_mean'].max().sort_values(ascending=False).nlargest(15)

state
New York                3962.34229
Michigan                3962.34229
Hawaii                  3962.34229
California              3962.34229
Florida                 3962.34229
Texas                   3962.34229
Tennessee               3962.34229
Virginia                3829.43383
District of Columbia    3661.71463
Connecticut             3600.28736
Missouri                3509.17617
Arizona                 3432.87803
New Jersey              3408.15112
Maryland                3341.37301
Pennsylvania            3324.65000
Name: rent_mean, dtype: float64

## Excluding records where rent_mean is 3962.34229 

In [12]:
re_data = re_data[re_data.rent_mean <= 3962]

# Part 2. Findings

## 2.1 Top 10 cities where people have the highest income left, but percentage of ownership is the lowest.

## Calculating income left.

In [13]:
re_data['income_left'] = re_data['family_mean'] - re_data['rent_mean']*12

In [14]:
inc_rent = re_data[['city', 'lat', 'lng', 'family_mean','rent_mean','pct_own','hc_mortgage_mean','income_left']].groupby('city').mean()

In [15]:
inc_rent.sort_values(['pct_own', 'income_left'], ascending = [True, False])

,lat,lng,family_mean,rent_mean,pct_own,hc_mortgage_mean,income_left
city,,,,,,,
Fort Hood,31.115478,-97.843359,54229.84425,1187.26747,0.00824,449.50000,39982.63461
Fort Stewart,31.876190,-81.599531,45136.66552,1181.23053,0.01240,2249.50000,30961.89916
Fort Dix,40.014556,-74.627018,109363.35805,2025.72648,0.01393,1787.00000,85054.64029
Ft Meade,39.118584,-76.735280,70433.74812,1942.58976,0.01465,1147.00000,47122.67100
Yermo,35.402696,-116.644250,62476.28492,1313.20643,0.01495,649.50000,46717.80776
...,...,...,...,...,...,...,...
Tinton Falls,40.297418,-74.101307,156338.64690,2412.09616,0.98475,3031.87157,127393.49298
Boxford,42.683108,-71.018330,153956.80076,1364.30787,0.98575,3015.26631,137585.10632
Marriottsville,39.318977,-76.967660,157966.86087,2954.72892,0.98658,2883.50466,122510.11383


In [16]:
re_data[['city', 'lat', 'lng', 'family_mean','rent_mean','pct_own','hc_mortgage_mean','income_left']].groupby('city').mean().nsmallest(2,'rent_mean')

,lat,lng,family_mean,rent_mean,pct_own,hc_mortgage_mean,income_left
city,,,,,,,
Revillo,44.873619,-96.552102,81301.30396,500.38186,0.89217,1269.38985,75296.72164
Flemingsburg,38.463259,-83.727673,71008.31228,500.74452,0.77952,1235.29781,64999.37804


In [17]:
import plotly.express as px

In [18]:
px.set_mapbox_access_token('pk.eyJ1IjoiYWxleHZ6bmsiLCJhIjoiY2sza3ZqNmJ6MGUwbzNkbjN3NXM3b2FuZCJ9.aDeGkoLyAC0eiXB0mq4a3g')
carshare = px.data.carshare()
fig = px.scatter_mapbox(mapdf, lat="lat", lon="lng",size='income_left', color="pct_own",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=10, zoom=2)
fig.show()

NameError: name 'mapdf' is not defined

## Let's be misogynst

In [124]:
import numpy as np

re_data

,state,city,zip_code,lat,lng,pop,male_pop,female_pop,rent_mean,rent_median,...,home_equity_second_mortgage,second_mortgage,home_equity,debt,hs_degree,male_age_mean,female_age_mean,pct_own,married,income_left
0,Alaska,Unalaska,99685,53.621091,-166.770979,4619,2725,1894,1366.24657,1405.0,...,0.00469,0.01408,0.02817,0.72770,0.82841,38.45838,32.78177,0.25053,0.47388,97935.24581
1,Alaska,Eagle River,99577,61.174250,-149.284329,3727,1780,1947,2347.69441,2351.0,...,0.03609,0.06078,0.07407,0.75689,0.94090,37.26216,38.97956,0.94989,0.52381,120469.37537
3,Alaska,Anchorage,99501,61.229560,-149.893037,1941,892,1049,943.79086,914.0,...,0.00000,0.00000,0.04469,0.78212,0.89274,35.81912,37.00750,0.20247,0.44428,67334.90414
4,Alaska,Anchorage,99504,61.217082,-149.767214,5981,3076,2905,1372.84472,1222.0,...,0.04083,0.04083,0.08514,0.67246,0.95351,34.13110,34.96611,0.56936,0.51034,64507.04066
5,Alaska,Anchorage,99504,61.217507,-149.744426,5476,2916,2560,1351.27532,1353.0,...,0.02897,0.02897,0.05164,0.74685,0.87941,29.23608,33.63770,0.45011,0.39429,68716.04329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39025,Wyoming,Jackson,83001,43.438652,-110.769450,7986,3960,4026,1136.08074,1111.0,...,0.03602,0.03602,0.11844,0.72772,0.94284,38.23101,35.90644,0.54379,0.50483,90261.56812
39026,Wyoming,Mountain View,82939,41.295517,-110.323344,6629,3395,3234,727.87310,673.0,...,0.01245,0.02645,0.05602,0.49689,0.93762,36.94758,36.41243,0.78639,0.52106,80150.42831
39027,Wyoming,Worland,82401,44.014369,-107.956379,2578,1264,1314,625.27917,630.0,...,0.05200,0.05200,0.07333,0.54533,0.84651,38.09749,42.03007,0.68378,0.47577,59669.75562
39028,Wyoming,Upton,82730,43.846213,-104.570020,3442,1998,1444,782.11689,728.0,...,0.00453,0.00453,0.02267,0.43155,0.90238,44.12667,44.18268,0.82450,0.54247,86037.70540


In [155]:
re_data.corr()

,zip_code,lat,lng,pop,male_pop,female_pop,rent_mean,rent_median,hc_mean,hc_mortgage_mean,...,home_equity_second_mortgage,second_mortgage,home_equity,debt,hs_degree,male_age_mean,female_age_mean,pct_own,married,income_left
zip_code,1.000000,-0.141997,-0.926883,0.073442,0.090314,0.054783,0.050808,0.044182,-0.258361,-0.040604,...,0.079000,0.072477,-0.099594,0.039506,-0.107492,-0.109889,-0.144528,-0.072080,0.009503,-0.066995
lat,-0.141997,1.000000,0.081879,-0.106457,-0.101125,-0.108473,-0.054917,-0.054818,0.174509,0.065185,...,0.053546,0.056545,0.189031,0.132797,0.210901,0.000704,-0.003108,0.073702,0.015020,0.130919
lng,-0.926883,0.081879,1.000000,-0.081320,-0.097705,-0.062900,-0.146117,-0.136721,0.192352,-0.075196,...,-0.101669,-0.102895,0.017288,-0.098965,0.085114,0.084130,0.119656,0.094240,-0.011268,0.024128
pop,0.073442,-0.106457,-0.081320,1.000000,0.984474,0.985223,0.154667,0.148506,0.048974,0.105180,...,0.092814,0.089632,0.106256,0.257168,0.038301,-0.199822,-0.203569,0.085845,0.166517,0.113378
male_pop,0.090314,-0.101125,-0.097705,0.984474,1.000000,0.939861,0.149388,0.143639,0.040873,0.102109,...,0.090115,0.086671,0.102273,0.249011,0.022221,-0.212423,-0.206081,0.091822,0.143106,0.113482
female_pop,0.054783,-0.108473,-0.062900,0.985223,0.939861,1.000000,0.155189,0.148810,0.055413,0.105030,...,0.092670,0.089839,0.106962,0.257431,0.052845,-0.181545,-0.195026,0.077443,0.184376,0.109883
rent_mean,0.050808,-0.054917,-0.146117,0.154667,0.149388,0.155189,1.000000,0.974799,0.593577,0.759383,...,0.127944,0.151622,0.420022,0.429121,0.341076,0.059711,0.021299,0.155939,0.255821,0.604374
rent_median,0.044182,-0.054818,-0.136721,0.148506,0.143639,0.148810,0.974799,1.000000,0.560183,0.720045,...,0.127271,0.150040,0.395996,0.415793,0.308879,0.040840,0.002980,0.146861,0.242384,0.558982
hc_mean,-0.258361,0.174509,0.192352,0.048974,0.040873,0.055413,0.593577,0.560183,1.000000,0.797477,...,0.013698,0.038072,0.349458,0.265938,0.348385,0.130624,0.100448,0.104536,0.197820,0.659553
hc_mortgage_mean,-0.040604,0.065185,-0.075196,0.105180,0.102109,0.105030,0.759383,0.720045,0.797477,1.000000,...,0.107112,0.143626,0.478567,0.380339,0.327870,0.106417,0.062613,0.076272,0.232284,0.720226


In [154]:
min_debt = re_data['debt'].min()
re_data.loc[(re_data['debt']==min_debt) | (re_data['debt']==re_data['debt'].max())]

,state,city,zip_code,lat,lng,pop,male_pop,female_pop,rent_mean,rent_median,...,home_equity_second_mortgage,second_mortgage,home_equity,debt,hs_degree,male_age_mean,female_age_mean,pct_own,married,income_left
10953,Iowa,Cedar Falls,50613,42.514983,-92.460782,5192,2058,3134,1022.20886,775.0,...,0.0,0.0,0.00000,0.01587,0.98261,21.70087,21.09711,0.08456,0.02864,57861.42023
21409,North Carolina,Charlotte,28273,35.134814,-80.933465,2152,1077,1075,1146.33964,1142.0,...,0.0,0.0,0.06827,0.97992,0.99560,33.41849,33.52651,0.24442,0.35261,85506.30880


In [125]:
total_pop = re_data.groupby(['city', 'state']).sum()
# male_pop = total_pop['male_pop']
# female_pop = total_pop['female_pop']
total_pop = total_pop[['male_pop',"female_pop"]]

total_pop['ratio'] = total_pop['male_pop'] / total_pop["female_pop"]
# ratio = male_pop / female_pop
# total_pop['ratio'] = ratio
# total_pop_male/female = total_pop[['city','male_pop',"female_pop", 'ratio']]
# total_pop_male/female
total_pop.sort_values('ratio')
total_pop.columns = ['total_male_pop', 'total_female pop', 'mf_ratio']
df = re_data.set_index(['city', 'state'])
df = total_pop.join(df)
df = df.reset_index()

In [126]:
# re_data[re_data.city=='Anchorage']

In [127]:
# df = df.drop(['male/female_ratio'], axis=1)

## male/female ratio of each state city

In [128]:
df.iloc[:,:4]

,city,state,total_male_pop,total_female pop
0,Abbeville,Alabama,2446,2415
1,Abbeville,Alabama,2446,2415
2,Abbeville,Louisiana,8354,8424
3,Abbeville,Louisiana,8354,8424
4,Abbeville,Louisiana,8354,8424
...,...,...,...,...
36843,Zionsville,Indiana,5727,5974
36844,Zoarville,Ohio,1780,1753
36845,Zolfo Springs,Florida,1772,1892
36846,Zumbrota,Minnesota,2775,2829


In [129]:
print(total_pop.index.names)
print(total_pop.columns)
print(age_df.index.names)
print(age_df.columns)

['city', 'state']
Index(['total_male_pop', 'total_female pop', 'mf_ratio'], dtype='object')
[None]
Index([                       'city',                       'state',
                    ('rend_median',),                ('rent_mean',),
                     ('livability',),                    'zip_code',
                               'lat',                         'lng',
                               'pop',                    'male_pop',
                        'female_pop',                   'rent_mean',
                       'rent_median',                     'hc_mean',
                  'hc_mortgage_mean',          'hc_mortgage_median',
                           'hi_mean',                   'hi_median',
                       'family_mean', 'home_equity_second_mortgage',
                   'second_mortgage',                 'home_equity',
                              'debt',                   'hs_degree',
                     'male_age_mean',             'female_age_mean',
    

## Median being the 50th percentile of a state city
if the Median is higher than the average rent then that would imply the city is still affordable in rent.

In [130]:
# total_pop = re_data.groupby(['city', 'state']).sum()
# afford_df = total_pop.loc[:,['rent_median','rent_mean']]
# afford_df['livability'] = afford_df['rent_median']/afford_df['rent_mean']
# afford_df.sort_values('livability',ascending=False, inplace=True)
# afford_df.columns = ['rent_median', 'rent_mean','livability']
# # age_df = age_df.set_index(['city','state'])
# afford_df = afford_df.join(total_pop)
# afford_df = afford_df.reset_index()
# afford_df
df['livability'] = df['rent_median']/df['rent_mean']

df[['city','state','livability']]

# age_df = age_df.loc[:,['state', 'pop']]

,city,state,livability
0,Abbeville,Alabama,1.008565
1,Abbeville,Alabama,0.948840
2,Abbeville,Louisiana,0.974945
3,Abbeville,Louisiana,0.985326
4,Abbeville,Louisiana,1.048867
...,...,...,...
36843,Zionsville,Indiana,0.899120
36844,Zoarville,Ohio,1.040907
36845,Zolfo Springs,Florida,1.035680
36846,Zumbrota,Minnesota,0.781373


## let's compare libability in rent to income left over

In [131]:
df[['mf_ratio', 'livability']]


,mf_ratio,livability
0,1.012836,1.008565
1,1.012836,0.948840
2,0.991690,0.974945
3,0.991690,0.985326
4,0.991690,1.048867
...,...,...
36843,0.958654,0.899120
36844,1.015402,1.040907
36845,0.936575,1.035680
36846,0.980912,0.781373


In [143]:
misogyny = df.loc[df['mf_ratio'] < df['livability'],['city','state','mf_ratio','livability']]
non_misogyny = df.loc[df['mf_ratio'] > df['livability'],['city','state','mf_ratio','livability']]
misogyny


,city,state,mf_ratio,livability
4,Abbeville,Louisiana,0.991690,1.048867
5,Abbeville,South Carolina,0.952282,1.110738
7,Aberdeen,Maryland,0.907214,1.050773
8,Aberdeen,Maryland,0.907214,0.971899
12,Aberdeen,North Carolina,0.949676,0.992241
...,...,...,...,...
36836,Zephyrhills,Florida,0.917377,1.038923
36838,Zephyrhills,Florida,0.917377,1.010598
36842,Zionsville,Indiana,0.958654,1.027228
36844,Zoarville,Ohio,1.015402,1.040907


In [144]:
non_misogyny

,city,state,mf_ratio,livability
0,Abbeville,Alabama,1.012836,1.008565
1,Abbeville,Alabama,1.012836,0.948840
2,Abbeville,Louisiana,0.991690,0.974945
3,Abbeville,Louisiana,0.991690,0.985326
6,Abbeville,South Carolina,0.952282,0.773542
...,...,...,...,...
36840,Zimmerman,Minnesota,1.130727,0.981406
36841,Zion,Illinois,0.880521,0.800046
36843,Zionsville,Indiana,0.958654,0.899120
36846,Zumbrota,Minnesota,0.980912,0.781373
